## Cortical structure experiments - 1

This script performs permutation testing experiments on cortical structures for each cluster group with respect to the rest.

This script generates the necessary files to run the freesurfer glmfit -sim command, with permutation testing, and save the results, for later analysis and visualization.

More info about the permutation procedure in freesurfer can be found in:
https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/MultipleComparisonsV6.0Perm

In [27]:
import numpy as np
import pandas as pd
import os

**Initialization and loading of variables**

In [28]:
# Parameters of the procedure
clusters = 4
rd_seed = 1714
n_perm = 1000

# thickness or volume
mode = "thickness"
DX = ['CN', 'MCI', 'AD']
C = ["C1", "C2", "C3", "C4"]

# Path of the generated clusters, after a run of cimlr-ad.py
cluster_path = "/homedtic/gmarti/EXPERIMENTS/CIMLR-AD/cimlr4/"      
df_cluster = pd.read_csv(cluster_path + 'cluster_data.csv')
df_cluster.reset_index(inplace=True)

**Prepare files for the concrete problem**

In [29]:
# Names of subjects
fsid_names = [x for x in map(lambda s: 'ADNI' + s[0:3] + 'S' + s[6:], 
                             df_cluster.PTID.values)]

# Support dataframe, with the info that we need to write
df_qdec = pd.DataFrame(data={'fsid': fsid_names,
                            'cluster': df_cluster.C.values,
                            'diagnosis': df_cluster.DX.values},
                       columns = ['fsid', 'cluster', 'diagnosis'])

# We want to test the thickness in the population of each cluster
# against the rest, so that we can observe differences across the 
# whole cluster.

# Create a fsgd file for each of the tests. In total, we have 4 experiments
# (one for each cluster)
fsgd_files = []
for c in C:
    # Create FSGD file
    # Only write the subjects that 
    fsgd_file = 'freefiles/adni_files_ex1_' + c + '.fsgd'
    fsgd_files.append(fsgd_file)
    with open(fsgd_file, 'w') as f:
        f.write('GroupDescriptorFile 1\n')  # First line, mandatory
        f.write('Title CIMLR-AD ex1-' + c + 'vs Rest\n')   # Title, optional
        # only two classes: either the cluster class, or rest
        f.write('Class ' + c + '\n')   # Title, optional
        f.write('Class Rest\n')   # Title, optional
        for subj in df_qdec.itertuples():
            subj_ptid = subj.fsid
            # Select only subjects in the cluster we are selecting
            if c == 'C' + str(int(subj.cluster)):
                cluster = 'C' + str(int(subj.cluster))
            else:
                cluster = 'Rest'
            f.write('Input ' + subj_ptid + ' ' + cluster + '\n')

# Contrast: same for all the 4 experiments
def write_contrast(name, matrix):
    with open(name, 'w') as f:
        f.write(matrix)  # First line, mandatory

# Mtx is actually the same for all subjects, as we are only comparing
# two groups
mtx = 'freefiles/ex1.mtx'
contrast = "+1 -1\n"
write_contrast(mtx, contrast)

**Run mri-prepoc with the selected subjects**

In [4]:
# For each experiment, run mri-preproc to prepare for the experiments.
# config variables:


# also, need to do lh and rh hemisferes
hemis = ["lh", "rh"]
for f in fsgd_files:
    name = os.path.splitext(os.path.basename(f))[0]
    for h in hemis:
        # build the command
        cmdline = ["mris_preproc", '--fsgd', f]
        # fwhm10 hard coded! (TODO: maybe don't hardcode this?)
        cmdline += ['--cache-in', mode + '.fwhm10.fsaverage']
        cmdline += ['--target', 'fsaverage']
        cmdline += ['--hemi', h]
        cmdline += ['--out', "processed_files/" + h + "_" + mode + "." + name + ".10.mgh"]
        # If out file exists, no need to run it again!
        if os.path.exists("processed_files/" + h + "_" + mode + "." + name + ".10.mgh"):
            print("processed_files/" + h + "_" + mode + "." + name + ".10.mgh Already exists!")
            continue
        else:
            # Run the experiment
            os.system(' '.join(cmdline))

processed_files/lh_thickness.adni_files_ex1_C1.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex1_C1.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex1_C2.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex1_C2.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex1_C3.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex1_C3.10.mgh Already exists!
processed_files/lh_thickness.adni_files_ex1_C4.10.mgh Already exists!
processed_files/rh_thickness.adni_files_ex1_C4.10.mgh Already exists!


**run mri-glmfit and correct for multiple comparisons**

In [5]:
hemis = ["lh", "rh"]
for f in fsgd_files:
    name = os.path.splitext(os.path.basename(f))[0]
    for h in hemis:
        # build the command
        cmdline = ["mri_glmfit"]
        cmdline += ['--y', "processed_files/" + h + "_" + mode + "." + name + ".10.mgh"]
        cmdline += ['--fsgd', f, 'dods']
        cmdline += ['--C', mtx]
        cmdline += ['--surf', 'fsaverage', h]
        cmdline += ['--cortex']
        cmdline += ['--glmdir', "glm_output/" + h + "_" + name + ".glmdir"]
        # Needed for later correction
        cmdline += ['--eres-save']
        
        # Run the experiment
        os.system(' '.join(cmdline))

In [6]:
n_perm = 1000
hemis = ["lh", "rh"]
for f in fsgd_files:
    name = os.path.splitext(os.path.basename(f))[0]
    for h in hemis:
        ## Correction for multiple comparisons
        # build the command
        cmdline = ["mri_glmfit-sim"]
        cmdline += ['--glmdir', "/homedtic/gmarti/CODE/simlr-ad/cortical_experiments/glm_output/" + h + "_" + name + ".glmdir"]
        # HARDCODED
        # info about parameters: 
        # n_perm is number of permutations
        # 4 is the range for clusters (in the surface)
        # abs is to take into account both + and - diff
        # (as we have no null hyp about either, we keep them both)
        cmdline += ['--perm', str(n_perm), '1.3', 'abs']
        # cmdline += ['--cache', '3.0', 'abs']
        # clusters with <0.05 will be highlighted
        cmdline += ['--cwp', '0.05']
        # Correct for both hemispheres with Bonferroni
        cmdline += ['--2spaces']
        # Run in parallel (4 nodes)
        # cmdline += ['--bg', '4']
        # Run the experiment
        print(' '.join(cmdline))
        os.system(' '.join(cmdline))

mri_glmfit-sim --glmdir /homedtic/gmarti/CODE/simlr-ad/cortical_experiments/glm_output/lh_adni_files_ex1_C1.glmdir --perm 10 3.0 abs --cwp 0.05 --2spaces


In [30]:
# Visualize the results
# lh or rh
h = 'lh'
# Number of experiments (max is len(fsgd_files))
print('number of experiments:' + str(len(fsgd_files)))
fsgd_number = 3
name = os.path.splitext(os.path.basename(fsgd_files[fsgd_number]))[0]
directory = "glm_output/" + h + "_" + name + ".glmdir/"
print(directory)

# path to the glm-fit-sim output
cluster_name = 'ex1/perm.th13.abs.sig.ocn.mgh'
annot_name = 'ex1/perm.th13.abs.sig.ocn.annot'

# Cluster annot
#cmdline = 'freeview --f $SUBJECTS_DIR/fsaverage/surf/' + h +\
#          '.inflated:overlay=' + directory + cluster_name + ':overlay_threshold=1.3:annot=' +\
#            directory + annot_name + ' -viewport 3d'

# If we want region annot
cmdline = 'freeview --f $SUBJECTS_DIR/fsaverage/surf/' + h +\
          '.inflated:overlay=' + directory + cluster_name + ':overlay_threshold=1.3:annot=' +\
            'aparc.annot:annot_outline=1 -viewport 3d'

# Also, print resume file of clusters to have extra information
resume_file = directory + 'ex1/perm.th13.abs.sig.cluster.summary'
with open(resume_file, 'r') as f:
    for line in f:
        print(line)

print(cmdline)
os.system(cmdline)

number of experiments:4
glm_output/lh_adni_files_ex1_C4.glmdir/
# Cluster Growing Summary (mri_surfcluster)

# $Id: mri_surfcluster.c,v 1.57.2.3 2016/11/17 18:19:42 zkaufman Exp $

# $Id: mrisurf.c,v 1.781.2.6 2016/12/27 16:47:14 zkaufman Exp $

# CreationTime 2018/10/17-00:23:43-GMT

# cmdline mri_surfcluster.bin --in /homedtic/gmarti/CODE/simlr-ad/cortical_experiments/glm_output/lh_adni_files_ex1_C4.glmdir/ex1/sig.mgh --mask /homedtic/gmarti/CODE/simlr-ad/cortical_experiments/glm_output/lh_adni_files_ex1_C4.glmdir/mask.mgh --cwsig /homedtic/gmarti/CODE/simlr-ad/cortical_experiments/glm_output/lh_adni_files_ex1_C4.glmdir/ex1/perm.th13.abs.sig.cluster.mgh --sum /homedtic/gmarti/CODE/simlr-ad/cortical_experiments/glm_output/lh_adni_files_ex1_C4.glmdir/ex1/perm.th13.abs.sig.cluster.summary --ocn /homedtic/gmarti/CODE/simlr-ad/cortical_experiments/glm_output/lh_adni_files_ex1_C4.glmdir/ex1/perm.th13.abs.sig.ocn.mgh --annot aparc --cwpvalthresh 0.05 --o /homedtic/gmarti/CODE/simlr-ad/corti

0